In [ ]:
# 크롤러를 만들기 전 필요한 도구들을 임포트합니다.
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import os

# 시작 시간과 끝 시간, 현재 하고 있는 시간 지정
start_date = '20231228'
end_date = '20231231'
doing_date = start_date
news_code = 101
start = 1 # url 크롤링 시작 페이지(default = 1)
end = 3 # url 크롤링 끝 페이지

# 뉴스 코드 리스트
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}

# 페이지 수, 카테고리, 날짜를 입력값으로 받습니다.
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}

# url 리스트받아오기
def make_urllist(page_start, page_end, code, date): 
  urllist = []
  first_url=''
  for i in range(page_start, page_end + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)
    # headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
    news = requests.get(url, headers=headers)

    # BeautifulSoup의 인스턴스 생성합니다. 파서는 html.parser를 사용합니다.
    soup = BeautifulSoup(news.content, 'html.parser')

    # CASE 1
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    # CASE 2
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
    if first_url == news_list[0]:
      print("breakpoint")
      break
    # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져오기
    for line in news_list:
      urllist.append(line.a.get('href'))
    for line in news_list:
      newsData.append({
      'link': line.a.get('href')
      })
    first_url = news_list[0]
  print('make_urllist', len(newsData))
  return urllist

# 시간단위로 크롤링해보기
def make_data(urllist, code):
  text_list = []
  date_list = []
  newsData.reverse()
  for news in newsData:
    news_url = news['link']
    res = requests.get(news_url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    try:
      news_content = soup.select_one("#dic_area").text.replace("\n","").replace("\t","")
    except:
      continue
    
    try:
      html_date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
      news_date = html_date.attrs['data-date-time']
    except AttributeError:
      news_date = soup.select_one("#content > div.end_ct > div > div.article_info > span > em")
    
    date_list.append(news_date)
    text_list.append(news_content)
    
  df = pd.DataFrame({'date': date_list, 'news': text_list})
  df['code'] = idx2word[str(code)]
  date_list[:5]
  print(len(df))
  # 중복된 데이터 제거
  df.drop_duplicates(subset=['news'], inplace=True)
  print('뉴스 기사의 개수: ',len(df))
  return df


# 끝 시간보다 작거나 같을 때 까지만 크롤링 진행
while(doing_date <= end_date):
  newsData = []
  # 경제 파트의 url 가져오기
  url_list = make_urllist(start, end, news_code, doing_date)
  # 가져온 파트의 뉴스 기사 가져오기
  data = make_data(url_list, news_code)
  
  print(doing_date)
  print('뉴스 기사의 개수: ',len(url_list))
  print('뉴스 데이터 개수: ',len(newsData))
  print(len(data))
  
  # 데이터프레임 파일을 csv 파일로 저장
  csv_path = str(news_code) + "-" + doing_date + ".csv"
  data.to_csv(csv_path, index=False)

  if os.path.exists(csv_path):
    print('{} File Saved!'.format(csv_path))
    
  # 날짜 +1 해주기
  doing = datetime.datetime.strptime(doing_date, '%Y%m%d')
  doing = doing + datetime.timedelta(days = 1)
  doing_date = datetime.datetime.strftime(doing, "%Y%m%d")